# Week 8

predict the ratings on the test set using the two models.

In [1]:
import surprise
from surprise import Dataset
from surprise import Reader
from surprise import KNNBasic
import pandas as pd
import pickle



In [2]:
# load test & train data
test_df = pd.read_csv('csv_files/preprocessed_test_video_games.csv')
train_df = pd.read_csv('csv_files/preprocessed_train_video_games.csv')
highly_rated_freq = pd.read_csv('csv_files/item_highly_rated_freq.csv')
# load the models
with open('best_knn_model.pkl', 'rb') as f:
    best_knn = pickle.load(f)
with open('best_svd_model.pkl', 'rb') as f:
    best_svd = pickle.load(f)
    

predict test on unrated/unrank item??

In [3]:
# make a prediction on the test set using both models
test_df['knn_pred'] = test_df.apply(lambda row: best_knn.predict(row['user_id'], row['item_id']).est, axis=1)
test_df['svd_pred'] = test_df.apply(lambda row: best_svd.predict(row['user_id'], row['item_id']).est, axis=1)

In [5]:
# measure the root mean squared error of the predictions 
from sklearn.metrics import root_mean_squared_error
knn_rmse = root_mean_squared_error(test_df['rating'], test_df['knn_pred'])
svd_rmse = root_mean_squared_error(test_df['rating'], test_df['svd_pred'])
print(f'KNN RMSE: {knn_rmse}')
print(f'SVD RMSE: {svd_rmse}')

KNN RMSE: 0.9790583906355597
SVD RMSE: 0.9383846760989479


In [ ]:
# my RMSE for both model is above 0.9 which is really bad
# what is the limitations of the RMSE metrics?  RMSE is sensitive to outliers and doesn't provide a relative measure of error.
# It also assumes that errors are normally distributed, which may not be the case in real-world data.

### discuss limitations of this metrics
this metrics accounts for outliers which there is not much and I don't remove so I don't prefer to use Root mean square error. assumes distribution is normal which may not be the case.

In [6]:
# measure MAE of the predictions
from sklearn.metrics import mean_absolute_error
knn_mae = mean_absolute_error(test_df['rating'], test_df['knn_pred'])
svd_mae = mean_absolute_error(test_df['rating'], test_df['svd_pred'])
print(f'KNN MAE: {knn_mae}')
print(f'SVD MAE:{svd_mae}')

KNN MAE: 0.6784451078221739
SVD MAE:0.6689177866277481


Now, we are interested not in whether the system properly predicts the rating
of these items, but rather whether the system gives the best recommendations
for each user. 

To evaluate this, generate the top-k (with k = 10) recommendation for each test user. Based on the top-k recommendation list generated for
each user, and using the test data split, compute:  
- Hit Rate, averaged across users.
- Precision@k, averaged across users.
- Mean Average Precision (MAP@k).
- Mean Reciprocal Rank (MRR@k).
- Coverage.

In [ ]:
# give TopPop recommendations for a user in test dataset
# top pop on test or train??
train_df['highly_rated'] = train_df['rating'] >= 3
item_highly_rated_freq = train_df.groupby('item_id')['highly_rated'].sum().reset_index()
item_highly_rated_freq.columns = ['item_id', 'item_highly_rated_freq']
# convert top 10 into a separate dataframe
test_top_pop_df = item_highly_rated_freq.sort_values('item_highly_rated_freq', ascending=False).head(10)
print(test_top_pop_df)

        item_id  item_highly_rated_freq
439  B0086VPUHI                     154
495  B00BN5T30E                     150
873  B07YBXFDYN                     146
487  B00BGA9WK2                     136
421  B007CM0K86                     123
592  B00KIWEMIG                     109
865  B07YBWT3PK                     104
875  B07YBXFF99                     103
319  B004HD55VK                     102
700  B014R4KYMS                     102


In [28]:
# generate top 10 recommendations for a user using both models & ground truth
for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]
    user_test_df = user_test_df.sort_values(by='knn_pred', ascending=False)
    print(f'Top 10 KNN recommendations for user {user_id}:')
    print(user_test_df[['item_id', 'knn_pred']].head(10))
    
    user_test_df = user_test_df.sort_values(by='svd_pred', ascending=False)
    print(f'Top 10 SVD recommendations for user {user_id}:')
    print(user_test_df[['item_id', 'svd_pred']].head(10))

    user_test_df = user_test_df.sort_values(by='rating', ascending=False)
    print(f'Top 10 ground truth recommendations for user {user_id}:')
    print(user_test_df[['item_id', 'rating']].head(10))

Top 10 KNN recommendations for user AFKKKKYLVYHA2FELEWZGVEW3TXHA:
         item_id  knn_pred
3982  B006PP404Q  4.858168
0     B00Z9TIGCG  4.633330
907   B007CM0K86  4.624269
1714  B000N5Z2L4  4.611445
3720  B009GE3WUQ  4.609287
108   B07DK1H3H5  4.599073
997   B00CJ9OTNE  4.595188
3767  B014T4J5HQ  4.592531
1947  B01GY35E8Y  4.507121
Top 10 SVD recommendations for user AFKKKKYLVYHA2FELEWZGVEW3TXHA:
         item_id  svd_pred
3720  B009GE3WUQ  4.922315
907   B007CM0K86  4.848988
1714  B000N5Z2L4  4.751310
1947  B01GY35E8Y  4.620006
3982  B006PP404Q  4.547424
3767  B014T4J5HQ  4.386221
0     B00Z9TIGCG  4.296327
997   B00CJ9OTNE  4.074203
108   B07DK1H3H5  3.839271
Top 10 ground truth recommendations for user AFKKKKYLVYHA2FELEWZGVEW3TXHA:
         item_id  rating
3720  B009GE3WUQ     5.0
907   B007CM0K86     5.0
1714  B000N5Z2L4     5.0
3767  B014T4J5HQ     5.0
0     B00Z9TIGCG     5.0
997   B00CJ9OTNE     5.0
1947  B01GY35E8Y     4.0
108   B07DK1H3H5     4.0
3982  B006PP404Q     1.0
Top

Interesting insight for the first user is that the models predicted differently but had the same top 10.
based on the top k recommendations, I can see that the two models are giving different recommendations for the same user. This could be due to the fact that KNN is based on user similarity while SVD is based on latent factors. The KNN model may be recommending items that similar users have liked, while the SVD model may be recommending items that have similar latent factors to the items the user has liked in the past.

#### Hit Rate
percentage of users who >= 1 relevant item in their top-k list
Range 0-1 
#### Precision @ k 
average what fraction of the top k recommendations are actually relevant?
#### Mean Reciprocal Rank
Average of (1/rank of first relevant items) across users
penalizes putting relevant items far down the list. 
#### Mean Average Precision
Average precision at each position where a relevant item appears 


In [29]:
# calcuate hit rate of the recommendations 
# we just need to compare the top 10 recommendations with the ground truth list for each user and calculate the hit rate
# I think it just checks if there exists any common item in the top 10 recommendations and the ground truth list, 
# if there is a common item then it counts as a hit, otherwise it counts as a miss. 
# Then we can calculate the hit rate by dividing the number of hits by the total number of users.
# compare with TopPop recommendations

hit_rate_knn = 0
hit_rate_svd = 0
hit_rate_top_pop = 0
total_users = 0
for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]
    knn_recs = set(user_test_df.sort_values(by='knn_pred', ascending=False)['item_id'].head(10))
    svd_recs = set(user_test_df.sort_values(by='svd_pred', ascending=False)['item_id'].head(10))
    top_pop_recs = set(test_top_pop_df['item_id'].tolist())
    ground_truth = set(user_test_df.sort_values(by='rating', ascending=False)['item_id'].head(10))
    
    hit_rate_knn += len(knn_recs.intersection(ground_truth)) / 10 # intersection find common items
    hit_rate_svd += len(svd_recs.intersection(ground_truth)) / 10
    hit_rate_top_pop += len(top_pop_recs.intersection(ground_truth)) / 10
    total_users += 1
hit_rate_knn /= total_users
hit_rate_svd /= total_users
hit_rate_top_pop /= total_users
print(f'KNN Hit Rate: {hit_rate_knn}')
print(f'SVD Hit Rate: {hit_rate_svd}')  
print(f'TopPop Hit Rate: {hit_rate_top_pop}') 


KNN Hit Rate: 0.45104391648667524
SVD Hit Rate: 0.45241180705542955
TopPop Hit Rate: 0.024910007199424187


In [30]:
# calculate Precision@10 
# which fraction of the top 10 recommendations are relevant (i.e. in the ground truth top 10)
# Define Relevance: Convert raw ratings into binary labels (e.g., a rating ≥ 3 is "relevant" (1), and anything lower is "not relevant" (0))
# or a specific user, count how many relevant items appear in the first k positions and divide that number by k
# inlcude TopPop recommendations in the evaluation as well
precision_at_10_knn = 0
precision_at_10_svd = 0
precision_at_10_top_pop = 0
total_users = 0
for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]
    knn_recs = set(user_test_df.sort_values(by='knn_pred', ascending=False)['item_id'].head(10))
    svd_recs = set(user_test_df.sort_values(by='svd_pred', ascending=False)['item_id'].head(10))
    top_pop_recs = set(test_top_pop_df['item_id'].tolist())
    ground_truth = set(user_test_df[user_test_df['rating'] >= 3]['item_id']) # relevant items with rating >= 3
    
    precision_at_10_knn += len(knn_recs.intersection(ground_truth)) / 10
    precision_at_10_svd += len(svd_recs.intersection(ground_truth)) / 10
    precision_at_10_top_pop += len(top_pop_recs.intersection(ground_truth)) / 10
    total_users += 1
precision_at_10_knn /= total_users
precision_at_10_svd /= total_users
precision_at_10_top_pop /= total_users
print(f'KNN Precision@10: {precision_at_10_knn}')
print(f'SVD Precision@10: {precision_at_10_svd}')
print(f'TopPop Precision@10: {precision_at_10_top_pop}') # something is wrong with TopPo precision
# it should be 1 becasue all top 10 items are relevant, but it is not, I think it is because of the way we calculate the precision, we need to divide by the number of relevant items instead of 10, 
# because if there are less than 10 relevant items then we are dividing by a larger number which will give us a lower precision.
# i am confused should this interection between top pop and grouth truth be high bec

KNN Precision@10: 0.41619870410366855
SVD Precision@10: 0.4164146868250509
TopPop Precision@10: 0.02361411087113043


In [31]:
# calculate Mean Average Precision (MAP@10) for both models
# include TopPop recommendations in the evaluation as well
ap_knn_sum = 0
ap_svd_sum = 0
ap_top_pop_sum = 0
total_users = 0

for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]

    # relevant items for this user (binary relevance definition)
    ground_truth = set(user_test_df[user_test_df['rating'] >= 3]['item_id'])
    G = len(ground_truth)

    knn_ranked = user_test_df.sort_values(by='knn_pred', ascending=False)['item_id'].head(10).tolist()
    svd_ranked = user_test_df.sort_values(by='svd_pred', ascending=False)['item_id'].head(10).tolist()
    top_pop_ranked = test_top_pop_df['item_id'].tolist()
    # AP for KNN
    if G == 0:
        ap_knn = 0
    else:
        hits = 0
        precision_sum = 0
        for rank, item in enumerate(knn_ranked, start=1):
            if item in ground_truth:
                hits += 1
                precision_sum += hits / rank
        ap_knn = precision_sum / G

    # AP for SVD
    if G == 0:
        ap_svd = 0
    else:
        hits = 0
        precision_sum = 0
        for rank, item in enumerate(svd_ranked, start=1):
            if item in ground_truth:
                hits += 1
                precision_sum += hits / rank
        ap_svd = precision_sum / G

    # AP for TopPop
    if G == 0:
        ap_top_pop = 0
    else:
        hits = 0
        precision_sum = 0
        for rank, item in enumerate(top_pop_ranked, start=1):
            if item in ground_truth:
                hits += 1
                precision_sum += hits / rank
        ap_top_pop = precision_sum / G

    ap_knn_sum += ap_knn
    ap_svd_sum += ap_svd
    ap_top_pop_sum += ap_top_pop
    total_users += 1

map_at_10_knn = ap_knn_sum / total_users
map_at_10_svd = ap_svd_sum / total_users
map_at_10_top_pop = ap_top_pop_sum / total_users
print(f'KNN MAP@10: {map_at_10_knn}')
print(f'SVD MAP@10: {map_at_10_svd}')
print(f'TopPop MAP@10: {map_at_10_top_pop}')


KNN MAP@10: 0.934798826700387
SVD MAP@10: 0.9450800670850975
TopPop MAP@10: 0.019034120668327867


MAP scale from 0 to 1, where 1 means perfect recommendations and 0 means no relevant items were recommended. A higher MAP indicates better performance of the recommendation system in ranking relevant items higher in the recommendation list.

In [32]:
# calculate mean repiprocal rank (MRR)
# For each user, find the rank of the first relevant item in the top 10 recommendations. The reciprocal rank is 1 divided by this rank. If there are no relevant items in the top 10, the reciprocal rank is 0. 
# Then, average these reciprocal ranks across all users to get the MRR.
# MRR is a scale between 0 and 1, where a higher MRR indicates better performance. 
# An MRR of 1 means that the first relevant item is always ranked at the top position (rank 1) for every user,
# while an MRR close to 0 indicates that relevant items are rarely ranked highly in the recommendations.
# include TopPop recommendations in the evaluation as well
mrr_knn = 0
mrr_svd = 0
mrr_top_pop = 0
total_users = 0
for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]
    knn_recs = user_test_df.sort_values(by='knn_pred', ascending=False)['item_id'].head(10)
    svd_recs = user_test_df.sort_values(by='svd_pred', ascending=False)['item_id'].head(10)
    top_pop_recs = test_top_pop_df['item_id'].tolist()
    ground_truth = set(user_test_df[user_test_df['rating'] >= 3]['item_id']) # relevant items with rating >= 3
    
    # Find first relevant item in KNN recommendations
    first_relevant_knn = None
    for i, item in enumerate(knn_recs):
        if item in ground_truth:
            first_relevant_knn = i + 1 # rank is 1-indexed
            break
    mrr_knn += 1 / first_relevant_knn if first_relevant_knn else 0

    # Find first relevant item in TopPop recommendations
    first_relevant_top_pop = None
    for i, item in enumerate(top_pop_recs):
        if item in ground_truth:
            first_relevant_top_pop = i + 1 # rank is 1-indexed
            break
    mrr_top_pop += 1 / first_relevant_top_pop if first_relevant_top_pop else 0
    
    # Find first relevant item in SVD recommendations
    first_relevant_svd = None
    for i, item in enumerate(svd_recs):
        if item in ground_truth:
            first_relevant_svd = i + 1 # rank is 1-indexed
            break
    mrr_svd += 1 / first_relevant_svd if first_relevant_svd else 0
    
    total_users += 1

mrr_knn /= total_users
mrr_svd /= total_users
mrr_top_pop /= total_users

print(f'KNN MRR: {mrr_knn}')
print(f'SVD MRR: {mrr_svd}')
print(f'TopPop MRR: {mrr_top_pop}')

KNN MRR: 0.9530477561795054
SVD MRR: 0.9635657718811066
TopPop MRR: 0.07091204132240848


In [34]:
# calculate coverage of the recommendations
# Catalog Coverage (CC@k): This is the most common metric for recommendation lists. 
# It is calculated by taking the total number of unique items that appear in the top-k lists for all users 
# and dividing that by the total number of items (n) in the entire catalog
# include SVD recommendations
# include top pop recommendations
all_recommended_items = set()
for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]
    # this needs to be recommend 
    all_recommended_items.update(user_test_df.sort_values(by='knn_pred', ascending=False)['item_id'].head(10).tolist())

Knn_catalog_coverage = len(all_recommended_items) / len(test_df['item_id'].unique())
print(f'Catalog Coverage KNN: {Knn_catalog_coverage}')

# Calculate SVD catalog coverage
all_recommended_items_svd = set()
for user_id in test_df['user_id'].unique():
    user_test_df = test_df[test_df['user_id'] == user_id]
    all_recommended_items_svd.update(user_test_df.sort_values(by='svd_pred', ascending=False)['item_id'].head(10).tolist())

Svd_catalog_coverage = len(all_recommended_items_svd) / len(test_df['item_id'].unique())
print(f'Catalog Coverage SVD: {Svd_catalog_coverage}')

# Calculate TopPop catalog coverage
all_recommended_items_top_pop = set()
all_recommended_items_top_pop.update(test_top_pop_df['item_id'].tolist())
TopPop_catalog_coverage = len(all_recommended_items_top_pop) / len(test_df['item_id'].unique())
print(f'Catalog Coverage TopPop: {TopPop_catalog_coverage}')

Catalog Coverage KNN: 0.9967637540453075
Catalog Coverage SVD: 0.9978425026968716
Catalog Coverage TopPop: 0.010787486515641856


the scale is between 0 and 1, where a higher CC@k indicates better coverage. A CC@k of 1 means that all items in the catalog are recommended to at least one user in the top-k lists, while a CC@k close to 0 indicates that only a small fraction of the catalog is being recommended.


Discuss the advantages and disadvantages of these metrics.
Coverage seems a bit basic only checking the percentage of items in the catalog that were recommended to at least 1 person. I feel like this does not really show how good our model performs on predictions.

Hit rate can shows how well our predicted top k list performs against the ground truth top k list across all users. 0.45 is quite low. I think comparing to ground truth top 10 is valuable metric.

precision@k: Is valuable in giving us insight into relevancy. ratings >=3 
does the top 10 actually provide high rated items to the user?

the other metrics like mean average precision and mean repricoal rank perform high. I think this is problematic where I score high on some metrics and then very low on the first two


# Analysis on the effect of Long Tail

In [ ]:
# sort the users by the number of ratings in the train set 
# and select the top 20% and last 20% of users
user_rating_counts = train_df.groupby('user_id').size()
sorted_users = user_rating_counts.sort_values(ascending=False) # descending
top_20_percent = sorted_users.head(int(len(sorted_users) * 0.2))
bottom_20_percent = sorted_users.tail(int(len(sorted_users) * 0.2))
print("number of users:", len(sorted_users))
print(f"Top 20% users: {len(top_20_percent)}")
print(f"Bottom 20% users: {len(bottom_20_percent)}")

number of users: 1389
Top 20% users: 277
Bottom 20% users: 277


In [39]:
# compute Hit rate on the test set for the 2 groups of users 
# and discuss the difference or similarity between the scores
# don't understand what were are comparing 

def compute_hit_rate(user_ids, test_df, pred_col):
    hit_rate = 0
    total_users = 0
    for user_id in user_ids:
        user_test_df = test_df[test_df['user_id'] == user_id]
        # get top 10 recommendations for the user based on the specified prediction column
        recs = set(user_test_df.sort_values(by=pred_col, ascending=False)['item_id'].head(10))
        ground_truth = set(user_test_df[user_test_df['rating'] >= 3]['item_id'])
        hit_rate += len(recs.intersection(ground_truth)) / 10
        total_users += 1
    return hit_rate / total_users
hit_rate_top_20_knn = compute_hit_rate(top_20_percent.index, test_df, 'knn_pred')
hit_rate_bottom_20_knn = compute_hit_rate(bottom_20_percent.index, test_df, 'knn_pred')
hit_rate_top_20_svd = compute_hit_rate(top_20_percent.index, test_df, 'svd_pred')
hit_rate_bottom_20_svd = compute_hit_rate(bottom_20_percent.index, test_df, 'svd_pred')
print(f'KNN Hit Rate for Top 20% users: {hit_rate_top_20_knn}')
print(f'KNN Hit Rate for Bottom 20% users: {hit_rate_bottom_20_knn}')
print(f'SVD Hit Rate for Top 20% users: {hit_rate_top_20_svd}')
print(f'SVD Hit Rate for Bottom 20% users: {hit_rate_bottom_20_svd}')

KNN Hit Rate for Top 20% users: 0.6844765342960271
KNN Hit Rate for Bottom 20% users: 0.269675090252707
SVD Hit Rate for Top 20% users: 0.6855595667870018
SVD Hit Rate for Bottom 20% users: 0.269675090252707


In [ ]:
# Sort the items by number of ratings (sorting by frequency) in the train set 
# and select the top 20% and last 20% of items
item_rating_counts = train_df.groupby('item_id').size()
sorted_items = item_rating_counts.sort_values(ascending=False) # descending
top_20_percent_items = sorted_items.head(int(len(sorted_items) * 0.2))
bottom_20_percent_items = sorted_items.tail(int(len(sorted_items) * 0.2))   
print("number of items:", len(sorted_items))
print(f"Top 20% items: {len(top_20_percent_items)}")
print(f"Bottom 20% items: {len(bottom_20_percent_items)}")

number of items: 932
Top 20% items: 186
Bottom 20% items: 186


In [ ]:
# compute the Coverage on the test set for the 2 groups of items and discuss differences and similarities
def compute_coverage(item_ids, test_df, pred_col):
    recommended_items = set()
    for user_id in test_df['user_id'].unique():
        user_test_df = test_df[test_df['user_id'] == user_id]
        recs = set(user_test_df.sort_values(by=pred_col, ascending=False)['item_id'].head(10))
        recommended_items.update(recs)
    coverage = len(recommended_items.intersection(set(item_ids))) / len(item_ids) 
    # the intersection of the set of items recommend and items in the top 20 or bottom 20, 
    # divided by the set of items in the group (top 20% or bottom 20%)
    return coverage
coverage_top_20_items = compute_coverage(top_20_percent_items.index, test_df, 'knn_pred')
coverage_bottom_20_items = compute_coverage(bottom_20_percent_items.index, test_df, 'knn_pred')
print(f'Coverage for Top 20% items: {coverage_top_20_items}')
print(f'Coverage for Bottom 20% items: {coverage_bottom_20_items}')
coverage_top_20_items_svd = compute_coverage(top_20_percent_items.index, test_df, 'svd_pred')
coverage_bottom_20_items_svd = compute_coverage(bottom_20_percent_items.index, test_df, 'svd_pred')
print(f'Coverage for Top 20% items (SVD): {coverage_top_20_items_svd}')
print(f'Coverage for Bottom 20% items (SVD): {coverage_bottom_20_items_svd}')
# basically how many items from the top 20% and bottom 20% are recommended by the model,
# and we can compare the coverage for both groups of items to 
# see if the model is biased towards recommending popular items (top 20%) or less popular items (bottom 20%).

Coverage for Top 20% items: 1.0
Coverage for Bottom 20% items: 1.0
Coverage for Top 20% items (SVD): 1.0
Coverage for Bottom 20% items (SVD): 1.0
